In [2]:
import os
import copy
import torch
from tqdm import tqdm
import pandas as pd
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [3]:
# 기존 데이터셋 삭제
dataset_name = 'coco_trash_final'
if dataset_name in DatasetCatalog:
    DatasetCatalog.remove(dataset_name)
if dataset_name in MetadataCatalog:
    MetadataCatalog.remove(dataset_name)

# Register Dataset
try:
    register_coco_instances('coco_trash_final', {}, '/data/ephemeral/home/data/dataset/test.json', '/data/ephemeral/home/data/dataset/')
except AssertionError:
    pass

# try:
#     register_coco_instances('coco_trash_final', {}, '/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/Split_data/valid_0_10.json', '/data/ephemeral/home/data/dataset/')
# except AssertionError:
#     pass

In [ ]:
from detectron2.engine import (
    AMPTrainer,
    SimpleTrainer,
    default_argument_parser,
    default_setup,
    default_writers,
    hooks,
    launch,
)
from detectron2.modeling import GeneralizedRCNNWithTTA, GeneralizedRCNN
import logging
from detectron2.engine.defaults import create_ddp_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import inference_on_dataset, print_csv_format
from detectron2.config import LazyConfig, instantiate
from omegaconf import OmegaConf
import yaml
import pandas as pd
from tqdm import tqdm
import torch
import os

# 설정 파일 로드 (LazyConfig)
cfg = LazyConfig.load('/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/config.yaml')
cfg.dataloader.test.dataset.names = 'coco_trash_final'

model = instantiate(cfg.model)
model.to(cfg.train.device)
model = create_ddp_model(model)
# 모델 가중치 로드
DetectionCheckpointer(model).load('/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/vitdet_rcnn_cascade/output/model_0049999.pth')

# 테스트 데이터셋 로드
test_loader = instantiate(cfg.dataloader.test)

# 모델을 평가 모드로 설정
model.eval()

# 예측 수행
prediction_strings = []
file_names = []

for data in tqdm(test_loader):
    
    prediction_string = ''
    input=data[0]
    with torch.no_grad():
        outputs = model(data)[0]['instances']  # model에 올바른 형식으로 전달
    
    # 예측 결과 처리
    targets = outputs.pred_classes.cpu().tolist()
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    scores = outputs.scores.cpu().tolist()
    
    for target, box, score in zip(targets, boxes, scores):
        prediction_string += (str(target) + ' ' + str(score) + ' ' + str(box[0]) + ' ' 
        + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ')
    
    prediction_strings.append(prediction_string)
    file_names.append(input['file_name'].replace('/data/ephemeral/home/data/dataset/', ''))

# 제출 파일 생성
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.train.output_dir, 'submission_det2.csv'), index=False)
